In [1]:
:set -XNoImplicitPrelude
:set -XOverloadedStrings
:set -XLambdaCase

In [2]:
import Papa
import Control.Monad.State
import qualified Prelude as P (read, undefined)
import qualified Data.IntMap as IM
import Data.Monoid
import Control.Monad.Reader



In [15]:
type Input = [Int]
type Output = IM.IntMap Int

data ParseResult' a =  Halt' Output | Result' Input a | Unknown' deriving (Show)

instance Functor ParseResult' where
    _ `fmap` Halt' a = Halt' a
    _ `fmap` Unknown' = Unknown'
    f `fmap` Result' i a = Result' i (f a)


isHalt' :: ParseResult' a -> Bool
isHalt' (Halt' _) = True
isHalt' _ = False

isError :: ParseResult' a -> Bool
isError Unknown' = True
isError _ = False

onResult :: ParseResult' a -> 
    (Input -> a -> ParseResult' b) -> 
    ParseResult' b
onResult Unknown' _ = Unknown'
onResult (Halt' i) _ = Halt' i
onResult (Result' i a) k = k i a

-- onHalt :: ParseResult' a -> Maybe Output
-- onHalt (Halt' i) = Just i
-- onHalt _ = Nothing

newtype Parser' a = Parser' (Input -> ParseResult' a) 

parse' :: Parser' a -> Input -> ParseResult' a
parse' (Parser' p) = p

instance Functor Parser' where
    f `fmap` (Parser' p) = Parser' (fmap f . p )

instance Applicative Parser' where
    pure a = Parser' (`Result'` a) -- valueParser
    
    (Parser' f) <*> (Parser' p) = Parser' (\i -> case p i of
        Halt' a -> Halt' a
        Unknown' -> Unknown'
        Result' j a -> (\g -> g a) <$> f j)
        
instance Monad Parser' where
    (Parser' p) >>= f = Parser' (\i ->
        onResult (p i) (\j a -> parse' (f a) j))
        
(|||) :: Parser' a -> Parser' a -> Parser' a
(Parser' p1) ||| (Parser' p2) = Parser' (\i -> let v = p1 i in bool v (p2 i) (isError v || isHalt' v))

list1 :: Parser' a -> Parser' [a]
list1 p = p >>= (\a -> (a :) <$> list p)

list :: Parser' a -> Parser' [a]
list p = list1 p ||| pure []

listt :: Parser' a -> Parser' [a]
listt = P.undefined


addParser :: Parser' (Input -> Output)
addParser = Parser' (
    \case
        l@(o : a : b : c : xs) | o == 1 -> 
            Result' xs $ 
                \j -> 
                    let v = IM.fromList $ itoList j
                    in
                        maybe 
                            v
                            ((`IM.union` v) . IM.singleton c) 
                            (liftA2 (liftA2 (+)) (!! a) (!! b) j)
        _ -> Unknown')
        
prodParser :: Parser' (Input -> Output)
prodParser = Parser' (
    \case
        l@(o : a : b : c : xs) | o == 2 -> 
            Result' xs $ 
                \j -> 
                    let v = IM.fromList $ itoList j
                    in
                        maybe 
                            v
                            ((`IM.union` v) . IM.singleton c) 
                            (liftA2 (liftA2 (*)) (!! a) (!! b) j)
        _ -> Unknown')        
    
    
-- prodParser :: Parser' Output
-- prodParser = haltParser ||| Parser' (
--     \case 
--         l@(o : a : b : c : xs) | o == 2 -> 
--             let v = IM.fromList $ itoList l
--             in
--             Result' l $ 
--                 maybe 
--                     v 
--                     ((`IM.union` v) . IM.singleton c) 
--                     (liftA2 (liftA2 (*)) (!! a) (!! b) l) 
--         _ -> Unknown')
    
haltParser :: Parser' (Input -> Output)
haltParser = Parser' (
    \case
        l@(o : _) | o == 99 -> Halt' (IM.fromList $ itoList l)
        _ -> Unknown')
        






    

In [14]:
let d = [1,9,10,3,2,3,11,0,99,30,40,50]
fmap (flip Papa.sequence d) $ parse' (list (addParser ||| prodParser ||| haltParser)) d

Result' [99,30,40,50] [fromList [(0,1),(1,9),(2,10),(3,70),(4,2),(5,3),(6,11),(7,0),(8,99),(9,30),(10,40),(11,50)],fromList [(0,150),(1,9),(2,10),(3,3),(4,2),(5,3),(6,11),(7,0),(8,99),(9,30),(10,40),(11,50)]]

In [70]:
:t Papa.sequence [(+1)] 2

Papa.sequence [(+1)] 2 :: forall t. Num t => [t]